In [28]:
# Midst re clustering, something is wrong
import pandas as pd
import json

macniece_annotation = "../data/dataset/temp/perspectrum_120618_after_lunch.json"
# original_annotation = "../data/dataset/temp/perspectrum_fixed.json"
original_annotation = "../data/dataset/perspectrum_with_answers_v0.1.json"
finished = "../data/dataset/temp/finished.csv"

ma = json.load(open(macniece_annotation))
oa = json.load(open(original_annotation))
fdf = pd.read_csv(finished).dropna()
fdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112 entries, 0 to 395
Data columns (total 4 columns):
cid1        112 non-null int64
cid2        112 non-null int64
equal       112 non-null object
finished    112 non-null object
dtypes: int64(2), object(2)
memory usage: 4.4+ KB


In [26]:
# Fix weird imparity (caused by previous faulty code)
from copy import deepcopy

get_all_changed_ones = set(list(fdf.cid1.unique()) + list(fdf.cid2.unique()))
# get_all_changed_ones = {}

oa_dict = {}
for oc in oa:
    oa_dict[oc["cId"]] = oc
    
wrong = 0
missing = 0
change_cid_list = []

for mc in ma:
    cid = mc["cId"]
    
    if cid not in oa_dict:
        missing += 1
    else:
        oc = oa_dict[cid]

        if cid not in get_all_changed_ones:
            if oc != mc:
                change_cid_list.append(cid)
                wrong += 1
                print(mc)

print("Missing: {}".format(missing))
print("Wrong: {}".format(wrong))
print("Wrong List: {}".format(change_cid_list))

{'cId': 3, 'text': 'Trade is better than aid', 'source': 'idebate', 'perspectives': [{'pids': [18, 20841, 20842], 'stance_label_3': 'SUPPORT', 'stance_label_5': 'SUPPORT', 'voter_counts': [4, 1, 0, 0, 0], 'evidence': [21, 4054, 18, 6078]}, {'pids': [19, 20843, 20844], 'stance_label_3': 'SUPPORT', 'stance_label_5': 'SUPPORT', 'voter_counts': [3, 2, 0, 0, 0], 'evidence': [19]}, {'pids': [76, 20846, 20915, 20916, 20], 'stance_label_3': 'SUPPORT', 'stance_label_5': 'SUPPORT', 'voter_counts': [4, 0, 0, 0, 1], 'evidence': [5739, 5741, 3278, 5774, 3280, 17, 20]}]}
{'cId': 2, 'text': 'Celebrities should be exempt from National Service (in South Korea)', 'source': 'idebate', 'perspectives': [{'pids': [20832, 20833, 20834, 20839, 20840, 10, 15, 24700], 'stance_label_3': 'UNDERMINE', 'stance_label_5': 'UNDERMINE', 'voter_counts': [0, 0, 0, 3, 0], 'evidence': [10, 15, 4308, 4310, 4312, 4313]}, {'pids': [14, 20835, 20836, 20837, 20838], 'stance_label_3': 'UNDERMINE', 'stance_label_5': 'UNDERMINE', 

In [21]:
# Applying the fixes and save'em
print("Claim numbers = {}".format(len(ma)))
print("Deleting faulty perspectives...")

for cid in change_cid_list:
    for mc in ma:
        if mc["cId"] == cid:
            ma.pop(ma.index(mc))
            break
    
print("After Claim numbers = {}".format(len(ma)))
print("Adding back correct perspectives...")

for cid in change_cid_list:
    ma.append(deepcopy(oa_dict[cid]))

print("Final Claim numbers = {}".format(len(ma)))
out_path = "../data/dataset/temp/perspectrum_fixed.json"
json.dump(ma, open(out_path, 'w'))

Claim numbers = 908
Deleting faulty perspectives...
After Claim numbers = 900
Adding back correct perspectives...
Final Claim numbers = 908


In [32]:
# Get the list of claims we haven't covered, and cluster them!
all_checked_ones = set(list(fdf.cid1.unique()) + list(fdf.cid2.unique()))

all_claims = set([oc["cId"] for oc in oa])

claims_remaining = all_claims - all_checked_ones

with open("../data/dataset/temp/remaining_claims", 'w') as fout:
    for cid in claims_remaining:
        fout.write(str(cid)+"\n")